### Ejercicio 23: Vinos

Un fabricante de vinos desea mezclar vinos de cinco años diferentes para fabricar vinos de
tres calidades diferentes (A, B y C). La cantidad de vino disponible de cada año (llamados año
1, 2, 3, 4, 5) es de 800, 900, 500, 900 y 600 barriles respectivamente. La calidad del vino A es tal
que como máximo se producirán 200 barriles. Se desea maximizar el beneficio.



#### Importamos Pyomo

In [35]:
from pyomo.environ import *
import pandas as pd

In [36]:
df = pd.DataFrame({'Beneficio':[4000,3000,2500]}, index= ['A','B','C'])
df2 =pd.DataFrame({'Disp':[800,900,500,900,600]},index=[1,2,3,4,5])

#### Creamos un problema concreto, es decir, un objeto de la clase problema que ya contenga parámetros

In [37]:
model = ConcreteModel()

#### Necesitamos utilizar sets

In [38]:
model.m=Set(initialize=df.index)
model.a=RangeSet(5)

#### Definimos las variables

In [39]:
model.Fin=Var(model.a, model.m, within=NonNegativeReals)

#### Definimos la función objetivo

In [40]:
model.obj = Objective(expr = sum(df['Beneficio'][m]*sum(model.Fin[a,m] for a in model.a) for m in model.m), sense=maximize)

#### Definimos la restricciones

In [41]:
model.disponib = ConstraintList()
for a in model.a:
    model.disponib.add(
        df2['Disp'][a]>=sum(model.Fin[a,m] for m in model.m)
    )

model.maxA = Constraint(expr= sum(model.Fin[a,'A'] for a in model.a)<=200)  

model.A1 = Constraint(expr=model.Fin[1,'A']+model.Fin[2,'A']>=0.6*sum(model.Fin[a,'A'] for a in model.a))
model.A2 = Constraint(expr=model.Fin[4,'A']+model.Fin[4,'A']<=0.1*sum(model.Fin[a,'A'] for a in model.a))
model.B = Constraint(expr=model.Fin[1,'B']+model.Fin[2,'B']+model.Fin[3,'B']>=0.5*sum(model.Fin[a,'B'] for a in model.a))
model.C = Constraint(expr=model.Fin[5,'C']<=0.5*sum(model.Fin[a,'C'] for a in model.a))



#### Resolvemos el problema

In [42]:
results = SolverFactory('glpk').solve(model)
model.pprint()
results.write()   

3 Set Declarations
    Fin_index : Size=1, Index=None, Ordered=True
        Key  : Dimen : Domain : Size : Members
        None :     2 :    a*m :   15 : {(1, 'A'), (1, 'B'), (1, 'C'), (2, 'A'), (2, 'B'), (2, 'C'), (3, 'A'), (3, 'B'), (3, 'C'), (4, 'A'), (4, 'B'), (4, 'C'), (5, 'A'), (5, 'B'), (5, 'C')}
    disponib_index : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    5 : {1, 2, 3, 4, 5}
    m : Size=1, Index=None, Ordered=Insertion
        Key  : Dimen : Domain : Size : Members
        None :     1 :    Any :    3 : {'A', 'B', 'C'}

1 RangeSet Declarations
    a : Dimen=1, Size=5, Bounds=(1, 5)
        Key  : Finite : Members
        None :   True :   [1:5]

1 Var Declarations
    Fin : Size=15, Index=Fin_index
        Key      : Lower : Value : Upper : Fixed : Stale : Domain
        (1, 'A') :     0 : 200.0 :  None : False : False : NonNegativeReals
        (1, 'B') :     0 : 600.0 :  None : False : False : No